In [ ]:
!sudo apt update
!sudo apt install tesseract-ocr

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,923 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [32.9 kB]
Get:13 https://ppa.la

In [ ]:
%pip install pytesseract

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, time, itertools, shutil, cv2, pytesseract
import numpy as np

from os.path import basename
from PIL import Image, UnidentifiedImageError
from sklearn.metrics.pairwise import cosine_similarity

import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import mobilenet_v3_small
from tqdm import tqdm
from transformers import CLIPModel, CLIPProcessor

# Define model and transform only once
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

_model = mobilenet_v3_small(pretrained=True)
_model = torch.nn.Sequential(*list(_model.children())[:-1])
_model.to(device).eval()

_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def remove_lower_res_duplicates(folder_path):
    """
    Removes duplicate images that have the same name but different extensions.
    Keeps the one with higher image dimensions (width*height).
    """
    valid_exts = {'.jpg', '.jpeg', '.png', '.webp', '.bmp', '.tiff'}
    files_by_name = {}

    # Group files by basename (without extension)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if not os.path.isfile(file_path):
            continue
        name, ext = os.path.splitext(filename)
        ext = ext.lower()
        if ext in valid_exts:
            files_by_name.setdefault(name.lower(), []).append(file_path)

    # Process groups
    for name, paths in files_by_name.items():
        if len(paths) > 1:  # Multiple extensions for same name
            best_file = None
            best_size = -1

            for path in paths:
                try:
                    with Image.open(path) as img:
                        width, height = img.size
                        size = width * height
                        if size > best_size:
                            best_size = size
                            best_file = path
                except Exception as e:
                    print(f"Error reading {path}: {e}")

            # Delete the rest
            for path in paths:
                if path != best_file:
                    try:
                        os.remove(path)
                        print(f"Deleted lower-res: {path}")
                    except Exception as e:
                        print(f"Error deleting {path}: {e}")

    print("Cleanup complete.")

def move_text_heavy_images(parent_dir, text_ratio_thresh=0.000035):
    text_heavy_dir = os.path.join(parent_dir, "text_heavy")
    os.makedirs(text_heavy_dir, exist_ok=True)

    valid_exts = ('.jpg', '.jpeg', '.png')
    image_files = [f for f in os.listdir(parent_dir) if f.lower().endswith(valid_exts)]

    for img_file in tqdm(image_files, desc="Scanning images", unit="img"):
        img_path = os.path.join(parent_dir, img_file)
        try:
            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            text = pytesseract.image_to_string(gray)
            text_len = len(text.strip())
            ratio = text_len / (img.shape[0] * img.shape[1])

            # Print ratio for each image
            tqdm.write(f"{img_file} --> Text Ratio: {ratio:.5f}")

            if ratio > text_ratio_thresh:
                shutil.move(img_path, os.path.join(text_heavy_dir, img_file))
        except Exception as e:
            tqdm.write(f"Skipping {img_file}: {e}")

class ImageDataset(Dataset):
    def __init__(self, image_paths, transform):
        self.image_paths = []
        for path in image_paths:
            try:
                with Image.open(path) as img:
                    img.verify()  # lightweight check
                self.image_paths.append(path)
            except (UnidentifiedImageError, OSError):
                print(f"[✗] Removing corrupted image: {path}")
                try:
                    os.remove(path)
                except Exception as e:
                    print(f"  Could not delete {path}: {e}")
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        try:
            image = Image.open(path).convert("RGB")
        except (UnidentifiedImageError, OSError):
            print(f"[✗] Skipping broken image during load: {path}")
            os.remove(path)
            return None
        return self.transform(image), path


def group_similar_images(input_dir, eps=0.13, batch_size=32, model=_model, transform=_transform, use_gpu=True):
    if model is None or transform is None:
        raise ValueError("Model and transform must be provided for optimized usage.")

    device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')

    valid_exts = ('.jpg', '.jpeg', '.png')
    image_paths = [os.path.join(input_dir, f)
                   for f in os.listdir(input_dir)
                   if f.lower().endswith(valid_exts)]

    if not image_paths:
        print("No valid images found.")
        return

    dataset = ImageDataset(image_paths, transform)
    if not len(dataset):
        print("[!] No valid images left after removing corrupted ones.")
        return

    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    embeddings, final_paths = [], []

    # Progress bar for embedding extraction
    print("\n🔍 Extracting embeddings...")
    for batch in tqdm(loader, total=len(loader), desc="Embedding Progress"):
        if batch is None:
            continue
        batch_imgs, batch_paths = batch
        batch_imgs = batch_imgs.to(device)
        with torch.no_grad():
            batch_emb = model(batch_imgs).squeeze(-1).squeeze(-1).cpu().numpy()
            batch_emb = batch_emb / np.linalg.norm(batch_emb, axis=1, keepdims=True)
        embeddings.append(batch_emb)
        final_paths.extend(batch_paths)

    embeddings = np.vstack(embeddings).astype(np.float32)

    used = set()
    cluster_id = 1
    n = len(final_paths)

    # Progress bar for clustering
    print("\n🔗 Clustering similar images...")
    for i in tqdm(range(n), desc="Clustering Progress"):
        if i in used:
            continue

        cluster_indices = [i]
        used.add(i)

        for j in range(i + 1, n):
            if j in used:
                continue
            sim = np.dot(embeddings[i], embeddings[j])
            if 1 - sim <= eps:
                cluster_indices.append(j)
                used.add(j)

        if len(cluster_indices) > 1:
            group_dir = os.path.join(input_dir, f"group_{cluster_id}")
            os.makedirs(group_dir, exist_ok=True)
            print(f"\n📂 Cluster {cluster_id}: {len(cluster_indices)} images")

            for a, b in itertools.combinations(cluster_indices, 2):
                sim = np.dot(embeddings[a], embeddings[b])
                print(f"  {basename(final_paths[a])} ↔ {basename(final_paths[b])} → sim: {sim:.4f}, dist: {1 - sim:.4f}")

            for idx in cluster_indices:
                shutil.move(final_paths[idx], os.path.join(group_dir, os.path.basename(final_paths[idx])))

            cluster_id += 1

    print(f"\n[✓] Grouped {len(used)} images into {cluster_id - 1} strict clusters.")

def pick_best_image_per_folder(parent_dir):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load CLIP model and processor
    _clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device).eval()
    _clip_proc = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    # Prompt can be tuned here
    text_prompt = "a beautiful, sharp, well-composed photo with attractive facial expressions, tall & slim body, aesthetic eyes, flawless skin & background."
    text_inputs = _clip_proc(text=[text_prompt], return_tensors="pt").to(device)
    with torch.no_grad():
        text_feats = _clip_model.get_text_features(
            input_ids=text_inputs["input_ids"],
            attention_mask=text_inputs["attention_mask"]
        )
        text_feats = text_feats / text_feats.norm(p=2, dim=-1, keepdim=True)

    # Loop over each subfolder
    for subfolder in sorted(os.listdir(parent_dir)):
        subfolder_path = os.path.join(parent_dir, subfolder)
        if not os.path.isdir(subfolder_path):
            continue

        print(f"\n📁 Evaluating folder: {subfolder}")
        image_scores = []
        image_paths = []

        # Collect all images in this subfolder
        for filename in sorted(os.listdir(subfolder_path)):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                path = os.path.join(subfolder_path, filename)
                try:
                    img = Image.open(path).convert("RGB")
                except:
                    print(f"  ⚠️ Skipping unreadable image: {filename}")
                    continue

                img_inputs = _clip_proc(images=img, return_tensors="pt").to(device)

                with torch.no_grad():
                    img_feats = _clip_model.get_image_features(pixel_values=img_inputs["pixel_values"])
                    img_feats = img_feats / img_feats.norm(p=2, dim=-1, keepdim=True)
                    score = float(cosine_similarity(img_feats.cpu(), text_feats.cpu()).squeeze())

                image_scores.append(score)
                image_paths.append(path)

                print(f"  📷 {filename} --> Score: {score:.4f}")

        # Mark the best image
        if image_scores:
            best_idx = int(torch.tensor(image_scores).argmax())
            best_path = image_paths[best_idx]
            base, ext = os.path.splitext(os.path.basename(best_path))
            best_name = os.path.join(os.path.dirname(best_path), f"best_{base}{ext}")
            os.rename(best_path, best_name)
            print(f"\n✅ Best image in '{subfolder}': {os.path.basename(best_name)} (Score: {image_scores[best_idx]:.4f})")

def move_best_and_clean(parent_dir):
    # Walk through all subdirectories
    for root, dirs, files in os.walk(parent_dir, topdown=False):
        # Skip parent directory
        if root == parent_dir:
            continue

        # Check if this is a last-level subdirectory (has no further subdirs)
        if not dirs:
            for file in files:
                if file.lower().startswith("best_"):
                    src_path = os.path.join(root, file)
                    dest_filename = file[len("best_"):]
                    dest_path = os.path.join(parent_dir, dest_filename)

                    # Avoid overwriting in parent dir
                    counter = 1
                    while os.path.exists(dest_path):
                        name, ext = os.path.splitext(dest_filename)
                        dest_path = os.path.join(parent_dir, f"{name}_{counter}{ext}")
                        counter += 1

                    shutil.move(src_path, dest_path)

            # Delete the entire subdirectory and its contents
            shutil.rmtree(root)

    print("[✓] Done moving 'best_' files and cleaning subdirectories.")

def revert(parent_dir):
    for root, dirs, files in os.walk(parent_dir, topdown=False):
        if root == parent_dir:
            continue  # Skip the top-level directory

        for file in files:
            # Remove all 'best_' prefixes if present
            new_name = file
            while new_name.startswith("best_"):
                new_name = new_name[len("best_"):]

            src_path = os.path.join(root, file)
            dest_path = os.path.join(parent_dir, new_name)

            # Handle name conflicts
            if os.path.exists(dest_path):
                base, ext = os.path.splitext(new_name)
                i = 1
                while True:
                    conflict_name = f"{base}_{i}{ext}"
                    conflict_path = os.path.join(parent_dir, conflict_name)
                    if not os.path.exists(conflict_path):
                        dest_path = conflict_path
                        break
                    i += 1

            shutil.move(src_path, dest_path)

        # Remove the now-empty subdirectory
        try:
            os.rmdir(root)
        except OSError:
            pass

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
input_dir = r"/content/drive/MyDrive/iPhone"
revert(input_dir)

In [ ]:
t1 = time.time()
print(f"Processing images in: {input_dir}")

remove_lower_res_duplicates(input_dir)

# print("Filtering text-heavy images...")
# move_text_heavy_images(input_dir)
# t2 = time.time()
# print(f"Text-heavy images moving took: {t2 - t1:.2f} seconds\n")

print("Grouping similar images...")
group_similar_images(input_dir)
t3 = time.time()
print(f"Grouping similar images took: {t3 - t1:.2f} seconds\n")

print("Picking best images from similar ones...")
pick_best_image_per_folder(input_dir)
t4 = time.time()
print(f"Picking best images took: {t4 - t3:.2f} seconds\n")
print(f"Total time taken: {(t4-t1)/60:.2f} mins")

# print("Cleaning up and moving best images...")
# move_best_and_clean(input_dir)
# t5 = time.time()
# print(f"Cleaning bad pics took: {t5 - t4:.2f} seconds\n")

Processing images in: /content/drive/MyDrive/iPhone
Grouping similar images...

🔍 Extracting embeddings...


Embedding Progress: 100%|██████████| 140/140 [11:40<00:00,  5.01s/it]



🔗 Clustering similar images...


Clustering Progress:   0%|          | 10/4454 [00:00<00:48, 91.13it/s]


📂 Cluster 1: 4 images
  IMG_2143.JPG ↔ IMG_2143.JPEG → sim: 0.9022, dist: 0.0978
  IMG_2143.JPG ↔ IMG_2142.JPEG → sim: 0.8798, dist: 0.1202
  IMG_2143.JPG ↔ IMG_2142.JPG → sim: 0.9476, dist: 0.0524
  IMG_2143.JPEG ↔ IMG_2142.JPEG → sim: 0.9219, dist: 0.0781
  IMG_2143.JPEG ↔ IMG_2142.JPG → sim: 0.8905, dist: 0.1095
  IMG_2142.JPEG ↔ IMG_2142.JPG → sim: 0.9254, dist: 0.0746

📂 Cluster 2: 2 images
  IMG_2197.JPG ↔ IMG_2204.JPG → sim: 0.8714, dist: 0.1286

📂 Cluster 3: 3 images
  IMG_2223.JPG ↔ IMG_2224.JPG → sim: 0.8800, dist: 0.1200
  IMG_2223.JPG ↔ IMG_2223.JPEG → sim: 0.8748, dist: 0.1252
  IMG_2224.JPG ↔ IMG_2223.JPEG → sim: 0.8351, dist: 0.1649

📂 Cluster 4: 2 images
  IMG_2252.JPG ↔ IMG_1468.JPG → sim: 0.8919, dist: 0.1081

📂 Cluster 5: 2 images
  IMG_2254.JPG ↔ IMG_2254.JPEG → sim: 0.8733, dist: 0.1267

📂 Cluster 6: 2 images
  IMG_2267.JPG ↔ IMG_2267.JPEG → sim: 0.8902, dist: 0.1098

📂 Cluster 7: 3 images
  IMG_2269.PNG ↔ IMG_E2269.JPG → sim: 0.8929, dist: 0.1071
  IMG_2269.PNG ↔

...


📂 Cluster 984: 2 images
  IMG_2771.JPEG ↔ IMG_2772.JPEG → sim: 0.9219, dist: 0.0781

📂 Cluster 985: 2 images
  IMG_E2875.JPG ↔ IMG_2875.JPEG → sim: 0.9997, dist: 0.0003

📂 Cluster 986: 2 images
  IMG_2838.JPEG ↔ IMG_2837.JPEG → sim: 0.9856, dist: 0.0144

📂 Cluster 987: 3 images
  IMG_2825.JPEG ↔ IMG_2824.JPEG → sim: 0.9505, dist: 0.0495
  IMG_2825.JPEG ↔ IMG_2823.JPEG → sim: 0.9419, dist: 0.0581
  IMG_2824.JPEG ↔ IMG_2823.JPEG → sim: 0.9738, dist: 0.0262

📂 Cluster 988: 2 images
  IMG_2834.JPEG ↔ IMG_2833.JPEG → sim: 0.9732, dist: 0.0268

📂 Cluster 989: 2 images
  IMG_2888.JPEG ↔ IMG_2887.JPEG → sim: 0.9682, dist: 0.0318

📂 Cluster 990: 6 images
  IMG_2932.JPG ↔ IMG_2933.JPG → sim: 0.9534, dist: 0.0466
  IMG_2932.JPG ↔ IMG_2934.JPG → sim: 0.9456, dist: 0.0544
  IMG_2932.JPG ↔ IMG_2933.JPEG → sim: 0.9532, dist: 0.0468
  IMG_2932.JPG ↔ IMG_2934.JPEG → sim: 0.9449, dist: 0.0551
  IMG_2932.JPG ↔ IMG_2932.JPEG → sim: 0.9998, dist: 0.0002
  IMG_2933.JPG ↔ IMG_2934.JPG → sim: 0.9633, dist: 0

Clustering Progress: 100%|██████████| 4454/4454 [00:22<00:00, 194.25it/s]


📂 Cluster 1005: 2 images
  IMG_3265.JPEG ↔ IMG_3264.JPEG → sim: 0.9976, dist: 0.0024

📂 Cluster 1006: 2 images
  IMG_3197.JPEG ↔ IMG_3198.JPEG → sim: 0.9205, dist: 0.0795

📂 Cluster 1007: 3 images
  IMG_3339.JPEG ↔ IMG_3340.JPEG → sim: 0.9857, dist: 0.0143
  IMG_3339.JPEG ↔ IMG_3337.JPEG → sim: 0.9273, dist: 0.0727
  IMG_3340.JPEG ↔ IMG_3337.JPEG → sim: 0.9313, dist: 0.0687

📂 Cluster 1008: 3 images
  IMG_3294.JPEG ↔ IMG_3293.JPEG → sim: 0.9684, dist: 0.0316
  IMG_3294.JPEG ↔ IMG_3295.JPEG → sim: 0.9756, dist: 0.0244
  IMG_3293.JPEG ↔ IMG_3295.JPEG → sim: 0.9579, dist: 0.0421

📂 Cluster 1009: 4 images
  IMG_3423.JPEG ↔ IMG_3422.JPEG → sim: 0.9874, dist: 0.0126
  IMG_3423.JPEG ↔ IMG_3424.JPEG → sim: 0.9201, dist: 0.0799
  IMG_3423.JPEG ↔ IMG_3425.JPEG → sim: 0.9105, dist: 0.0895
  IMG_3422.JPEG ↔ IMG_3424.JPEG → sim: 0.9203, dist: 0.0797
  IMG_3422.JPEG ↔ IMG_3425.JPEG → sim: 0.9133, dist: 0.0867
  IMG_3424.JPEG ↔ IMG_3425.JPEG → sim: 0.9836, dist: 0.0164

📂 Cluster 1010: 2 images
  IM


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Streaming output truncated to the last 5000 lines.
📁 Evaluating folder: group_390
  📷 IMG_1812.JPG --> Score: 0.2014
  📷 IMG_1813.JPG --> Score: 0.1939

✅ Best image in 'group_390': best_IMG_1812.JPG (Score: 0.2014)

📁 Evaluating folder: group_391
  📷 IMG_1840.JPG --> Score: 0.1778
  📷 IMG_1855.JPG --> Score: 0.1638
  📷 IMG_1856.JPG --> Score: 0.1702

✅ Best image in 'group_391': best_IMG_1840.JPG (Score: 0.1778)

📁 Evaluating folder: group_392
  📷 IMG_1841.JPG --> Score: 0.1792
  📷 IMG_1842.JPG --> Score: 0.1690

✅ Best image in 'group_392': best_IMG_1841.JPG (Score: 0.1792)

📁 Evaluating folder: group_393
  📷 IMG_1810.JPG --> Score: 0.1681
  📷 IMG_1811.JPG --> Score: 0.1873

✅ Best image in 'group_393': best_IMG_1811.JPG (Score: 0.1873)

📁 Evaluating folder: group_394
  📷 IMG_1808.JPEG --> Score: 0.1855
  📷 IMG_1808.JPG --> Score: 0.1705
  📷 IMG_1809.JPG --> Score: 0.1902

✅ Best image in 'group_394': best_IMG_1809.JPG (Score: 0.1902)

📁 Evaluating folder: group_395
  📷 IMG_1762.JPG 

In [ ]:
from google.colab import files

folder_path = "/content/drive/MyDrive/iPhone"

# Create zip file (shutil will include the entire folder)
shutil.make_archive("/content/iPhone_backup", 'zip', folder_path)

FileNotFoundError: Cannot find file: /content/iPhone.zip

In [ ]:
zip_path = "/content/iPhone_backup.zip"

# Download the zip file
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print("Cleaning up and moving best images...")
move_best_and_clean(input_dir)

Cleaning up and moving best images...
[✓] Done moving 'best_' files and cleaning subdirectories.


## Past operations --> for optimizations only

In [ ]:
def print_cluster_similarities(image_paths, labels, embeddings):
    label_to_indices = {}
    for idx, label in enumerate(labels):
        if label == -1:
            continue  # Skip noise
        label_to_indices.setdefault(label, []).append(idx)

    for label, indices in label_to_indices.items():
        if len(indices) < 2:
            continue  # Skip singleton clusters

        print(f"\nCluster {label+1} — {len(indices)} images")
        for i, j in itertools.combinations(indices, 2):
            emb_i, emb_j = embeddings[i], embeddings[j]
            sim = np.dot(emb_i, emb_j)  # cosine similarity (L2-normalized)
            print(f"  {basename(image_paths[i])} ↔ {basename(image_paths[j])}  →  similarity: {sim:.4f}, distance: {1-sim:.4f}")

In [ ]:
# Run clustering
t1 = time.time()
group_similar_images(dir, eps=0.1, batch_size=32, model=_model, transform=_transform)
print(f"Completed in {time.time() - t1:.2f} seconds.")


📂 Cluster 1: 2 images
  IMG_20250621_165922782.jpg ↔ IMG_20250621_170008508.jpg → sim: 0.9244, dist: 0.0756

📂 Cluster 2: 3 images
  IMG_20250621_122354245.jpg ↔ IMG_20250621_122348827.jpg → sim: 0.9072, dist: 0.0928
  IMG_20250621_122354245.jpg ↔ IMG_20250621_122324444.jpg → sim: 0.9159, dist: 0.0841
  IMG_20250621_122348827.jpg ↔ IMG_20250621_122324444.jpg → sim: 0.9158, dist: 0.0842

📂 Cluster 3: 27 images
  IMG_20250718_135847115.jpg ↔ IMG_20250718_135747072.jpg → sim: 0.9663, dist: 0.0337
  IMG_20250718_135847115.jpg ↔ IMG_20250718_135820431.jpg → sim: 0.9119, dist: 0.0881
  IMG_20250718_135847115.jpg ↔ IMG_20250718_135814548.jpg → sim: 0.9146, dist: 0.0854
  IMG_20250718_135847115.jpg ↔ IMG_20250718_135812948.jpg → sim: 0.9056, dist: 0.0944
  IMG_20250718_135847115.jpg ↔ IMG_20250718_135815851.jpg → sim: 0.9121, dist: 0.0879
  IMG_20250718_135847115.jpg ↔ IMG_20250718_135808168.jpg → sim: 0.9722, dist: 0.0278
  IMG_20250718_135847115.jpg ↔ IMG_20250718_135806960.jpg → sim: 0.963

In [ ]:
t1 = time.time()
pick_best_image_per_folder(dir)
print(time.time()-t1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]


📁 Evaluating folder: group_1
  📷 IMG_20250621_165922782.jpg --> Score: 0.1320
  📷 IMG_20250621_170008508.jpg --> Score: 0.1540

✅ Best image in 'group_1': best_IMG_20250621_170008508.jpg (Score: 0.1540)

📁 Evaluating folder: group_10
  📷 IMG_20250722_101949160.jpg --> Score: 0.1822
  📷 IMG_20250722_101953933.jpg --> Score: 0.2022

✅ Best image in 'group_10': best_IMG_20250722_101953933.jpg (Score: 0.2022)

📁 Evaluating folder: group_11
  📷 IMG_20250621_164514759.jpg --> Score: 0.1674
  📷 IMG_20250621_164519122.jpg --> Score: 0.1682

✅ Best image in 'group_11': best_IMG_20250621_164519122.jpg (Score: 0.1682)

📁 Evaluating folder: group_12
  📷 IMG_20250628_204801095.jpg --> Score: 0.1916
  📷 IMG_20250628_204904721.jpg --> Score: 0.2062

✅ Best image in 'group_12': best_IMG_20250628_204904721.jpg (Score: 0.2062)

📁 Evaluating folder: group_13
  📷 IMG_20250621_122059083.jpg --> Score: 0.2060
  📷 IMG_20250621_122059990.jpg --> Score: 0.2154

✅ Best image in 'group_13': best_IMG_20250621_12